Requirements


In [ ]:
# specific to colab
!pip install datasets
!pip install gensim
from google.colab import drive, files
drive.mount('/content/drive')

# general imports
from datasets import load_dataset
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')
import pandas as pd
import string, re
from gensim.models import Word2Vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Loading the dataset

In [ ]:
# Load the TLDR-17 dataset from the webis repository.
dataset = load_dataset("webis/tldr-17", trust_remote_code=True)
print(dataset)
print(dataset["train"][0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3848330 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/38 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['author', 'body', 'normalizedBody', 'subreddit', 'subreddit_id', 'id', 'content', 'summary'],
        num_rows: 3848330
    })
})
{'author': 'raysofdarkmatter', 'body': "I think it should be fixed on either UTC standard or UTC+1 year around, with the current zone offsets.\n\nMoving timescales add a lot of complexity to the implementation of timekeeping systems and have [dubious value]( \n\nI think seasonal shifting time made sense in the pre-electric past, when timekeeping was more flexible and artificial light was inefficient and often dangerous. \n\nNow we have machines that work easily with simple timekeeping rules, and it's more beneficial to spend a small amount on energy for lighting, and save the larger cost of engineering things to work with the complex timekeeping rules, as well as saving the irritation to humans.\n\nLighting has gotten much more efficient over time; we can squeeze out a lot more photons per unit of energy 

Look into the subreddits

In [ ]:
# Extract the 'subreddit' field from the 'train' split of the dataset.
subreddits = dataset['train']['subreddit']

subreds = {}
# Iterate through each subredit to count them
for i in subreddits:
    if i in subreds:
        subreds[i] += 1
    else:
        subreds[i] = 1

# creating a list that consists of tuples with subreddits that have more than
# 500 posts
min_subreds = [(subred, posts) for subred, posts in subreds.items() if posts > 10000]
list_subreds = [subred for (subred, i) in min_subreds]
print(min_subreds)
print(list_subreds)

[('funny', 40171), ('atheism', 43268), ('AskReddit', 589947), ('personalfinance', 14403), ('leagueoflegends', 109307), ('videos', 20567), ('trees', 47286), ('Games', 11820), ('relationship_advice', 50416), ('TwoXChromosomes', 21562), ('AdviceAnimals', 40783), ('explainlikeimfive', 25482), ('WTF', 25781), ('starcraft', 12911), ('todayilearned', 25004), ('Guildwars2', 10948), ('technology', 13700), ('worldnews', 22577), ('DotA2', 22405), ('pics', 35098), ('talesfromtechsupport', 11537), ('relationships', 352049), ('politics', 36518), ('gaming', 43851), ('sex', 28806), ('wow', 13309), ('IAmA', 22689), ('buildapc', 14817), ('magicTCG', 10624), ('Drugs', 10035), ('dating_advice', 12361), ('NoFap', 13203), ('loseit', 10559), ('AskMen', 14538), ('summonerschool', 13806), ('tifu', 52219), ('askscience', 12037), ('Fitness', 22694), ('electronic_cigarette', 10430), ('movies', 10420), ('news', 11660), ('offmychest', 17175), ('depression', 11365), ('self', 10166), ('GlobalOffensive', 10443), ('pcm

Put the subreddit that we will use into a dataframe



In [ ]:
def extract_subred(dataset, subreddit):
    content = list()
    for content in dataset.filter(lambda dataset: dataset['subreddit'] == subreddit)['train']['content']:
      content.append(content)
    return content


# Extract the 'content' section of each subreddit and convert it to a CSV file
d = {"AskReddit": extract_subred(dataset, "AskReddit"), "relationship_advice": extract_subred(dataset, "relationship_advice"), "league of legends": extract_subred(dataset, "leagueoflegends"), "tifu": extract_subred(dataset, "tifu")}

df = pd.Series(d).to_frame()

df.to_csv("content_df")

Filter:   0%|          | 0/3848330 [00:00<?, ? examples/s]

KeyboardInterrupt: 

Extract the content of each subreddit and separate them to then tokenize separetly

In [ ]:
ds = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/content_df.csv', index_col="Unnamed: 0")

print(ds)

askreddit_content = ds.loc["AskReddit"]["0"]
relation_content = ds.loc["relationship_advice"]["0"]
league_content = ds.loc["league of legends"]["0"]
tifu_content = ds.loc["tifu"]["0"]

                                                                     0
AskReddit            Yeah, but most folks think avoiding gluten wil...
relationship_advice  She's been honest with you the entire time. An...
league of legends    Didn't they lose 6 games in a row? Just becaus...
tifu                 Harken all to my tale of dream pissing, for I ...


Tokenizer

In [ ]:
initial_ok_list = ("okay", "okiii", "okayyyy", "okey", "O.K", "okeh", "kk", "okie", "ok", "Ok", "k", "oki", "okiii", "okayy")

def custom_tokenizer(raw_list):
  # split into sentences and words: list of lists - maybe split into posts instead
  tokenized = sent_tokenize(raw_list)
  tokenized = [word_tokenize(i) for i in tokenized]
  # remove all tokens that are puntiation
  tokenized = [[i for i in sent if i not in string.punctuation] for sent in tokenized ]
  # for all tokens that are not a version of OK, make lowercase and remove puntuation
  tokenized = [[i if i in initial_ok_list else i.lower() for i in sent] for sent in tokenized ]
  tokenized = [[i if i in initial_ok_list else re.sub(r'[^\w\s]', '', i)  for i in sent] for sent in tokenized ]

  return tokenized

tok_askred = custom_tokenizer(askreddit_content[:200000000])
# tok_relation = custom_tokenizer(relation_content)
# tok_league = custom_tokenizer(league_content)
# tok_tifu = custom_tokenizer(tifu_content)

print(tok_askred[:50])
#print(tok_relation)
#print(tok_league)
# print(tok_tifu[:50])

[['yeah', 'but', 'most', 'folks', 'think', 'avoiding', 'gluten', 'will', 'cause', 'them', 'to', 'become', 'thin'], ['thats', 'why', 'i', 'call', 'it', 'a', 'fad'], ['a', 'woman', 'i', 'know', 'once', 'fell', 'asleep', 'after', 'eating', 'a', 'large', 'cheeseburger'], ['she', 'believed', 'her', 'sleep', 'was', 'caused', 'by', 'the', 'gluten', 'in', 'the', 'bun', 'and', 'now', 'she', 'goes', 'around', 'telling', 'people', 'shes', 'borderlime', 'celiacs'], ['funny', 'thing', 'is', 'i', 'have', 'seen', 'her', 'unknowingly', 'eat', 'foods', 'packed', 'with', 'gluten', 'since', 'the', 'whole', 'cheeseburger', 'incident', 'without', 'any', 'troubleyou', 'probably', 'wo', 'nt', 'come', 'off', 'as', 'an', 'ass', 'if', 'you', 'just', 'say', 'whats', 'on', 'your', 'mind', 'to', 'the', 'girl', 'you', 'like'], ['if', 'the', 'other', 'girl', 'gets', 'offended', 'that', 'you', 'like', 'the', 'other', 'chick', 'well', 'its', 'not', 'really', 'anyones', 'fault', 'but', 'her', 'own'], ['or', 'talk', 'ab

Testing content

In [ ]:
def test_content(tok_list):
  word = tok_list[0] #print the first word
  doc_type = type(tok_list) #print the type of content
  length = len(tok_list) #return the lenght of the content
  period = [i for i in tok_askred[:50] if i == "."] #return empty
  return [word, doc_type, length, period]

test_content(tok_askred)
# test_content(tok_relation)
# test_content(tok_league)
# test_content(tok_tifu)


[['yeah',
  'but',
  'most',
  'folks',
  'think',
  'avoiding',
  'gluten',
  'will',
  'cause',
  'them',
  'to',
  'become',
  'thin'],
 list,
 1987920,
 []]

Get all the different versions of OK found in all four subreddits.

In [ ]:
# Iterate through every item in 'content' looking for the OKs in the initial_ok_list

def get_ok(content):
  ok_words = list()
  for x in content:
    for i in x:
      if i in initial_ok_list:
        ok_words.append(i)
  return set(ok_words) # Return a unique set of OKs found in the 'content'

print(get_ok(tok_askred))
#tifu_oks = get_ok(tok_tifu))
#print(get_ok(tok_league))
#print(get_ok(tok_relation))

{'ok', 'okey', 'okay', 'Ok', 'okie', 'O.K', 'okayyyy', 'oki', 'kk', 'k'}


Word embedding model taken from https://www.geeksforgeeks.org/word-embeddings-in-nlp/



In [ ]:
def skipgram_w2v_model(tokenized_content, model_name):
	skipgram_model = Word2Vec(vector_size=100, # size of 2D vector
														window=5,		 # distance between the current and predicted word in each direction
														sg=1,			 # Skip-Gram model
														min_count=5)	 # Ignores all words with a total frequency lower than five

	# store all words in a vocabulary
	skipgram_model.build_vocab(tokenized_content)
	# Training
	skipgram_model.train(tokenized_content, total_examples=skipgram_model.corpus_count, epochs=10)
 	# saving the model
	skipgram_model.save(model_name)
  # Download the model: function may be specific to colab
	files.download(model_name)
	loaded_model = Word2Vec.load(model_name)

	return loaded_model

# tifu_model = skipgram_w2v_model(tok_tifu, "tifu_model.model")
askreddit_model = skipgram_w2v_model(tok_askred, "askreddit_model.model")
# relation_model = skipgram_w2v_model(tok_relation, "relation_model.model")
# league_model = skipgram_w2v_model(tok_league, "league_model.model")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>